In [27]:
# Training loop:
# 1. Create torch dataset from the data containing the query, positive, negative example. 
# 2. Create a dataloader from the dataset.
# 3. Run a single batch through the model.

In [28]:
!poetry add pandas

912.54s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
The following packages are already present in the pyproject.toml and will be skipped:

  • pandas

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [29]:
from torch.utils.data import DataLoader, Dataset
import pandas as pd

In [30]:
## Load data
train = pd.read_parquet('train.parquet')
test = pd.read_parquet('test.parquet')
validate = pd.read_parquet('validate.parquet')


In [92]:
print(train.shape, test.shape, validate.shape)
sorted(train['query_id'].values)

(82326, 6) (9650, 6) (10047, 6)


[19699,
 19700,
 19701,
 19702,
 19703,
 19704,
 19705,
 19706,
 19707,
 19708,
 19709,
 19710,
 19711,
 19712,
 19713,
 19714,
 19715,
 19716,
 19717,
 19718,
 19719,
 19720,
 19721,
 19722,
 19723,
 19724,
 19725,
 19726,
 19727,
 19728,
 19729,
 19730,
 19731,
 19732,
 19733,
 19734,
 19735,
 19736,
 19737,
 19738,
 19739,
 19740,
 19741,
 19742,
 19743,
 19744,
 19745,
 19746,
 19747,
 19748,
 19749,
 19750,
 19751,
 19752,
 19753,
 19754,
 19755,
 19756,
 19757,
 19758,
 19759,
 19760,
 19761,
 19762,
 19763,
 19764,
 19765,
 19766,
 19767,
 19768,
 19769,
 19770,
 19771,
 19772,
 19773,
 19774,
 19775,
 19776,
 19777,
 19778,
 19779,
 19780,
 19781,
 19782,
 19783,
 19784,
 19785,
 19786,
 19787,
 19788,
 19789,
 19790,
 19791,
 19792,
 19793,
 19794,
 19795,
 19796,
 19797,
 19798,
 19799,
 19800,
 19801,
 19802,
 19803,
 19804,
 19805,
 19806,
 19807,
 19808,
 19809,
 19810,
 19811,
 19812,
 19813,
 19814,
 19815,
 19816,
 19817,
 19818,
 19819,
 19820,
 19821,
 19822,
 19823,


In [55]:
full_dataset = pd.concat([train, test, validate])

In [96]:
total_size = train.shape[0] + test.shape[0] + validate.shape[0]
print(total_size)
# Rerun this code only if you want to generate new indices.
import numpy as np
np.random.seed(seed=999999)
indices = np.random.randint(0, total_size, total_size)
# train_indices = indices[0:train.shape[0]]
# test_indices = indices[train.shape[0]:train.shape[0]+test.shape[0]]
# validate_indices = indices[train.shape[0]+test.shape[0]:]
# all_indices = {'train': train_indices.tolist(), 'test': test_indices.tolist(), 'validate': validate_indices.tolist()}
# print(f"Length of all indices: {sum(len(v) for k, v in all_indices.items())}")
# # write these to a pickle file 
import pickle
# with open('indices.pkl', 'wb') as f:
#     pickle.dump(all_indices, f)

# create train, test, validation datasets based on query_ids
# query_ids = full_dataset['query_id']
# train_query_ids = query_ids[0:train.shape[0]]
# test_query_ids = query_ids[train.shape[0]:train.shape[0]+test.shape[0]]
# validate_query_ids = query_ids[train.shape[0]+test.shape[0]:]
# print(train_query_ids.shape, test_query_ids.shape, validate_query_ids.shape)
# all_query_ids  = {'train': train_query_ids.tolist(), 'test': test_query_ids.tolist(), 'validate': validate_query_ids.tolist()}
# # write these to a pickle file

# with open('query_ids.pkl', 'wb') as f:
#     pickle.dump(all_query_ids, f)




102023
(82326,) (9650,) (10047,)


In [99]:
# Unpickle and test 
with open('indices.pkl', 'rb') as f:
    all_indices = pickle.load(f)

    print(f"{len(all_indices['train'])} training indices {all_indices['train'][0:10]}")
    print(f"{len(all_indices['test'])} test indices {all_indices['test'][0:10]}")
    print(f"{len(all_indices['validate'])} validate indices {all_indices['validate'][0:10]}")

with open('query_ids.pkl', 'rb') as f:
    all_query_ids = pickle.load(f)
    print(f"{len(all_query_ids['train'])} training query ids {all_query_ids['train'][0:10]}")
    print(f"{len(all_query_ids['test'])} test query ids {all_query_ids['test'][0:10]}")
    print(f"{len(all_query_ids['validate'])} validate query ids {all_query_ids['validate'][0:10]}")

82326 training indices [36233, 13668, 19591, 66699, 34957, 34882, 23273, 27926, 90079, 68436]
9650 test indices [27928, 5317, 97860, 83225, 90861, 42316, 42625, 98527, 74133, 95026]
10047 validate indices [94011, 72401, 60227, 70263, 35765, 47026, 31472, 3385, 21289, 545]
82326 training query ids [19699, 19700, 19701, 19702, 19703, 19704, 19705, 19706, 19707, 19708]
9650 test query ids [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
10047 validate query ids [9652, 9653, 9654, 9655, 9656, 9657, 9658, 9659, 9660, 9661]


In [79]:

#train[['passages', 'query']].iloc[0]
train_dataset = full_dataset.iloc[all_indices['train']]
test_dataset = full_dataset.iloc[all_indices['test']]
validate_dataset = full_dataset.iloc[all_indices['validate']]

train_dataset.head()['query'].iloc[0]

'is there a limit to roth ira contributions'

In [88]:
print(f"dataset shape: {full_dataset.shape}")
all_urls = full_dataset['passages'].apply(lambda x: x['url']).tolist()
unique_urls = set([item for sublist in all_urls for item in sublist])
print(f"Total number of urls: {sum(len(i) for i in all_urls)}")
print(f"Total number of unique urls: {len(unique_urls)}")
ids_to_urls = {i: url for i, url in enumerate(unique_urls)}
urls_to_ids = {url: i for i, url in ids_to_urls.items()}

queries = full_dataset['query'].tolist()
assert len(queries) == len(set(queries))
print(f"Total number of queries: {len(queries)}")
queries_to_ids = 
"""
{query_id: (relevant_urls, irrelevant_urls)}
"""
for row in train_dataset.iloc[0:3].iterrows():

    print(row[1])

dataset shape: (102023, 6)
Total number of urls: 837729
Total number of unique urls: 456487
Total number of queries: 102023
Total number of unique queries: 102023
answers                                                          [Yes]
passages             {'is_selected': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]...
query                       is there a limit to roth ira contributions
query_id                                                         55966
query_type                                                 description
wellFormedAnswers                                                   []
Name: 36233, dtype: object
answers              [To relieve fever an pain., It is relieve mode...
passages             {'is_selected': [0, 0, 1, 0, 0, 0, 0, 0, 0], '...
query                              what is hydrocodone liquid used for
query_id                                                         33374
query_type                                                 description
wellFormedAnswers            

In [82]:
queries[0]

'what is rba'

In [ ]:

def create_triplet_dataset(which: str):
    """
    Structure: 
    query, positive, negative
    """
    pass


In [65]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.load('spm_AllTexts.model')

True

In [ ]:
class TripletDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]